In [1]:
import nltk

In [2]:
import numpy

In [3]:
import sklearn

In [4]:
from nltk.stem import PorterStemmer

In [5]:
ps = PorterStemmer()

In [6]:
from nltk.corpus import wordnet as wn

In [7]:
from nltk.parse.stanford import StanfordParser

In [8]:
from nltk.tree import ParentedTree, Tree

In [9]:
import os
 
os.environ['STANFORD_PARSER'] = '/usr/local/Cellar/stanford-parser/3.8.0/libexec'
os.environ['STANFORD_MODELS'] = '/usr/local/Cellar/stanford-parser/3.8.0/libexec'

In [10]:
parser = StanfordParser()

In [11]:
res = list(parser.raw_parse("What is your name?"))[0]

In [12]:
res = ParentedTree.convert(res)

In [13]:
from nltk import pos_tag,word_tokenize

In [102]:
def find_subject(t):
  for s in t.subtrees(lambda t: t.label() == 'NP'):
    for n in s.subtrees(lambda n: n.label().startswith('NN')):
      if(n != None):
          return (n[0], find_attrs(n))
    for n in s.subtrees(lambda n: n.label().startswith('PRP')):
      if(n != None):
          return (n[0], find_attrs(n))
    
def findQuestionMatter(t):
    ans = []
    for s in t.subtrees(lambda t: t.label().startswith("WH")):
      for n in s.subtrees(lambda n: n.label().startswith('WHADJP')):
        for n in s.subtrees((lambda n: n.label() == ('JJ'))):
          ans.append(n[0])
      for n in s.subtrees((lambda n: n.label().startswith('NN'))):
        if(n not in ans):
            ans.append(n)
      for n in s.subtrees(lambda n: n.label().startswith('WH')):
        for n in s.subtrees((lambda n: n.label() == ('WRB'))):
          ans.append(n)
        for n in s.subtrees((lambda n: n.label().startswith('WP'))):
          ans.append(n)
        for n in s.subtrees((lambda n: n.label().startswith('WD'))):
          ans.append(n)
    return ans
 
# Depth First Search the tree and take the last verb in VP subtree.
def find_predicate(t):
  v = None
 
  for s in t.subtrees(lambda t: t.label() == 'VP'):
    for n in s.subtrees(lambda n: n.label().startswith('VB')):
      v = n
  if(v != None):
      return (v[0], find_attrs(v))
 
# Breadth First Search the siblings of VP subtree
# and take the first noun or adjective
def find_object(t):
  for s in t.subtrees(lambda t: t.label() == 'VP'):
    for n in s.subtrees(lambda n: n.label() in ['NP', 'PP', 'ADJP']):
      if n.label() in ['NP', 'PP']:
        for c in n.subtrees(lambda c: c.label().startswith('NN')):
          return (c[0], find_attrs(c))
      else:
        for c in n.subtrees(lambda c: c.label().startswith('JJ')):
          return (c[0], find_attrs(c))
 
def find_attrs(node):
  attrs = []
  p = node.parent()
 
  # Search siblings of adjective for adverbs
  if node.label().startswith('JJ'):
    for s in p:
      if s.label() == 'RB':
        attrs.append(s[0])
 
  elif node.label().startswith('NN'):
    for s in p:
      if s.label() in ['DT','PRP$','POS','JJ','CD','ADJP','QP','NP']:
        attrs.append(s[0])

  elif node.label().startswith('NP'):
    for s in p:
      if s.label() in ['PRP$','NN']:
        attrs.append(s[0])
 
  # Search siblings of verbs for adverb phrase
  elif node.label().startswith('VB'):
    for s in p:
      if s.label() == 'ADVP':
        attrs.append(' '.join(s.flatten()))
 
  # Search uncles
  # if the node is noun or adjective search for prepositional phrase
  if node.label().startswith('JJ') or node.label().startswith('NN'):
      for s in p.parent():
        if s != p and s.label() == 'PP':
          attrs.append(' '.join(s.flatten()))
 
  elif node.label().startswith('VB'):
    for s in p.parent():
      if s != p and s.label().startswith('VB'):
        attrs.append(' '.join(s.flatten()))
 
  return attrs

In [95]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


In [96]:
print(find_attrs(res))

[]


In [108]:
def parseSentence(sentence):
    import re
    stopwords = ["are","is","am","was","were","a","an","the","do","did","Did"]
    
    replace = {
        'many' : 'number',
        'I' : 'me',
        'my' : 'me'
    }
    
    stopwords = set(stopwords)
    res = list(parser.raw_parse(sentence))[0]
    print(res)
    res = ParentedTree.convert(res)
    res.pretty_print()
    subjects = find_subject(res)
    if(subjects == None):
        subjects = ("",[])
    objects = find_object(res)
    if(objects == None):
        objects = ("",[])
    verbs = find_predicate(res)
    if(verbs == None):
        verbs = ("",[])
    attributes = find_attrs(res)
    if(attributes == None):
        attributes = ("",[])
    ques = findQuestionMatter(res)
    if(ques == None):
        ques = ("",[])
    print(subjects)
    print(objects)
    print(verbs)
    print(attributes)
    print(ques)
    tags = pos_tag(word_tokenize(sentence))
    print(pos_tag(word_tokenize(sentence)))
    
    sent = ""
    for s in subjects[1]:
        if(s not in stopwords):
            sent = sent + " " + s + " "
    if(subjects[0] not in sent):
        sent = sent + " " + subjects[0] + " "
    for o in objects[1]:
        if(o not in stopwords and o not in sent):
            sent = sent + " " + o + " "
    if(objects[0] not in sent and objects[0] not in stopwords):
        sent = sent + " " + ps.stem(objects[0]) + " "
    
    for v in verbs[1]:
        if(v not in stopwords):
            sent = sent + " " + ps.stem(v) + " "
    if(verbs[0] not in sent and ps.stem(verbs[0]) not in stopwords):
        sent = sent + " " + ps.stem(verbs[0]) + " "
    for q in ques:
        if(q[0] not in stopwords and q[0] not in set(sent)):
            sent = sent + " " + q[0] + " "
    for t in tags:
        if(t[1] == "JJS"):
            sent = sent + " " + t[0] + " "
    return re.sub( '\s+', ' ', sent ).strip()

In [110]:
parseSentence("I am from Himachal")

(ROOT
  (S (NP (PRP I)) (VP (VBP am) (PP (IN from) (NP (NNP Himachal))))))
        ROOT                  
         |                     
         S                    
  _______|____                 
 |            VP              
 |    ________|____            
 |   |             PP         
 |   |         ____|_____      
 NP  |        |          NP   
 |   |        |          |     
PRP VBP       IN        NNP   
 |   |        |          |     
 I   am      from     Himachal

('I', [])
('Himachal', [])
('am', [])
[]
[]
[('I', 'PRP'), ('am', 'VBP'), ('from', 'IN'), ('Himachal', 'NNP')]


'I himach'

In [24]:
ps.stem("office")

'offic'

In [20]:
lemmatizer.lemmatize("ran")

'ran'

In [21]:
print("a")

a


In [26]:
import sys

In [28]:
files = os.listdir("../vocab/")

In [34]:
print(type(files))

<class 'list'>


In [40]:
import csv
file = 'vocab.csv'
with open(file, 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for val in files:
        wr.writerow([val])

In [62]:
import en

ModuleNotFoundError: No module named 'article'

In [77]:
import difflib

In [82]:
from autocorrect import spell

ModuleNotFoundError: No module named 'autocorrect'

In [111]:
ps.stem("was")

'wa'